In [ ]:
def textpuller(text,version):
    link = "https://www.sefaria.org/api/texts/"+text+"?vhe="+version+"&context=0"
    print(link)
    with urllib.request.urlopen(link) as url:
        sefariaresult = json.loads(url.read().decode())
    text = ""
    if type(sefariaresult["he"])==str:
        text = sefariaresult["he"]
    elif type(sefariaresult["he"])==list:
        if type(sefariaresult["he"][0])==str:
            for verse in sefariaresult["he"]:
                text = text + verse + " "
        elif type(sefariaresult["he"][0])==list:
            textblock = []
            for chunk in sefariaresult["he"]:
                textblock = textblock + chunk
            for verse in textblock:
                text = text + verse + " "
    title_he = sefariaresult["heSectionRef"]
    return text,title_he

In [ ]:
def qkparse(string):
    start = string.index('<span class=\"mam-kq">')
    string_before = string[0:start]
    startk = string.index('<span class=\"mam-kq-k\">',start)
    startk = string.index('>',startk)+2
    endk = string.index('</span>',startk)-1
    ktext = string[startk:endk]
    startq = string.index('<span class=\"mam-kq-q\">')
    startq = string.index('>',startq)+2
    endq = string.index('</span>',startq)-1
    qtext=string[startq:endq]
    end = string.index('</span></span>',endq)
    end = string.index(' ',end)
    output = '\\qk{'+qtext+'}{'+ktext+'}'
    string_after = string[end:]
    return string_before+output+string_after

In [ ]:
def removeformatting(text):
    while "<" in text and ">" in text:
        loc1 = text.find("<")
        loc2 = text.find(">",loc1)+1
        text = text.replace(text[loc1:loc2],"")
    return text